In [2]:
import gc
import numpy as np
import pandas as pd
from keras import backend as K
from keras.callbacks import CSVLogger, ModelCheckpoint, Callback
from sklearn.model_selection import ParameterGrid
import datetime
import os
import pickle

from models import MDAD_model
from experiment_helpers import load_final_PCA_data, save_MTL_predictions

os.environ["CUDA_VISIBLE_DEVICES"]="2"
K.tensorflow_backend._get_available_gpus()

['/job:localhost/replica:0/task:0/device:GPU:0']

In [3]:
hyperparams = {"epochs": [200], \
               "nonlinearity": ["relu"], \
               "hidden_sizes_shared": [[500,100]], \
               "hidden_sizes_separate": [[50,10]],\
               "dropout":  [.1],\
               "k_reg": [.00001,.001],\
#                 "learning_rate": [.001],\
            "learning_rate": [.0001,.001],\
               "loss_weights":  [[1, 1]],\
               "grad_clip_norm": [.01,.1],\
               "batch_size": [20]}
hy_dict_list = list(ParameterGrid(hyperparams))
len(hy_dict_list)

8

In [6]:
SPECIFIC_FOLDER = "origGE"

dataset ="ACT_MSBBRNA_ROSMAP_PCA"

path_to_all = "../../md-ad_public_repo_data/Modeling/final_model/%s/"%SPECIFIC_FOLDER
path_to_results = path_to_all + "results/MTL/"
path_to_models = path_to_all + "models/MTL/"
     
MTL_final_final_model = pickle.load(open("../../AD_Project/analyses/MTL_variable_tasks/6vars-continuous/%s/final_models_chosen/MTL/final.p"%SPECIFIC_FOLDER, "rb" ) )

for hy_iteration in range(len(hy_dict_list)):

    hy_dict = hy_dict_list[hy_iteration]

    title = "%d_%s_%s_%s_%f_%f_%f_%s_%f_%d"%(hy_dict["epochs"], hy_dict["nonlinearity"], 
                    str(hy_dict["hidden_sizes_shared"]), str(hy_dict["hidden_sizes_separate"]),
                    hy_dict["dropout"], hy_dict["k_reg"], hy_dict["learning_rate"], 
                    str(hy_dict["loss_weights"]),  hy_dict["grad_clip_norm"], hy_dict["batch_size"])
    print(title)
    if title == MTL_final_final_model:
        print("FINAL SELECTED HY ID:", hy_iteration)
        FINAL_HY = hy_iteration



for rep in range(100):
    print(rep)
    
    X,y = load_final_PCA_data(SPECIFIC_FOLDER)

    print(X.shape)

    for hy_iteration in [FINAL_HY]:

        print("HYPERPARAMETER ITERATION: %d"%hy_iteration)
    
        hy_dict = hy_dict_list[hy_iteration]

        title = "%d_%s_%s_%s_%f_%f_%f_%s_%f_%d"%(hy_dict["epochs"], hy_dict["nonlinearity"], 
                        str(hy_dict["hidden_sizes_shared"]), str(hy_dict["hidden_sizes_separate"]),
                        hy_dict["dropout"], hy_dict["k_reg"], hy_dict["learning_rate"], 
                        str(hy_dict["loss_weights"]),  hy_dict["grad_clip_norm"], hy_dict["batch_size"])
        print(title)

        res_dest = path_to_results +  "/" + dataset + "/" + title + "/" + str(rep) + "/"
        modelpath = path_to_models + dataset + "/" + title +  "/" + str(rep) + "/"
        for path in [res_dest, modelpath]:
            if not os.path.isdir(path):
                os.makedirs(path)
            
            
        model = MDAD_model(X, hy_dict)    

        csv_logger = CSVLogger(res_dest+'results.log')

        History = model.fit(x={'main_input': X},\
                            y={'BRAAK_out': y["BRAAK"], 'CERAD_out': y["CERAD"], \
                               'PLAQUES_out': y["PLAQUES"], 'TANGLES_out': y["TANGLES"],
                   "ABETA_IHC_out": y["ABETA_IHC"], "TAU_IHC_out":y["TAU_IHC"]}, \
                  verbose=0,epochs=hy_dict["epochs"], batch_size=hy_dict["batch_size"], callbacks=[csv_logger,\
                ModelCheckpoint(modelpath+"{epoch:02d}.hdf5", monitor='val_loss', verbose=0, \
                save_best_only=False, save_weights_only=False, mode='auto', period=200)])


        K.clear_session()
        gc.collect()